In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.optim import Adam
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [2]:
composed = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
composed_ext = transforms.Compose([transforms.RandomHorizontalFlip(), 
                                   transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                                   transforms.RandomRotation(degrees=20),
                                   transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                                   transforms.ToTensor()
                                   ])
train = dsets.CIFAR10(root='./data', train=True, download=True, transform=composed_ext)
test = dsets.CIFAR10(root='./data', train=False, download=True, transform=composed)

100%|██████████| 170498071/170498071 [00:01<00:00, 102880134.55it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
x_train = train.data
y_train = train.targets
x_test = test.data
y_test = test.targets

In [16]:
model = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(128),
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(128),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(128 * 8 * 8, 4000),
        nn.BatchNorm1d(4000),
        nn.LeakyReLU(),
        nn.Dropout(0.5),
        nn.Linear(4000 , 1000),
        nn.LeakyReLU(),
        nn.Dropout(0.5),
        nn.Linear(1000 , 200),
        nn.BatchNorm1d(200),
        nn.LeakyReLU(),
        nn.Dropout(0.5),
        nn.Linear(200, 50),
        nn.LeakyReLU(),
        nn.Dropout(0.5),
        nn.Linear(50,10),
        nn.LeakyReLU()
)

In [17]:
optimizer = Adam(model.parameters(),lr=0.01, betas=(0.9,0.99))

loss_func = nn.CrossEntropyLoss()

def batches(input , target , batchsize = 8):
    for i in range(0,input.shape[0],batchsize):
        yield(input[i : i + batchsize],target[i : i + batchsize])

In [18]:
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available!")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

CUDA is available!


In [22]:

accuracy = 0
num = 0
for epoch in range(0,50):
    accuracy = 0
    num = 0
    model.train()
    for X,Y in batches(x_train,y_train,256):
        X = torch.Tensor(X.transpose(0, 3, 1, 2)) # Convert X to a PyTorch Tensor
        Y = torch.Tensor(Y)
        X = X.to(device)
        Y = Y.to(device)
        prediction = model(X)
        loss = loss_func(prediction , Y.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        accuracy += (prediction.max(1)[1] == Y).sum().item()
        num += Y.size(0)
    print(accuracy/num)

0.98646
0.98932
0.98826
0.98828
0.9901
0.98638
0.98974
0.9884
0.98974
0.98776
0.9909
0.99138
0.98804
0.99118
0.9906
0.9887
0.99114
0.99006
0.98866
0.99078
0.99132
0.9907
0.98962
0.99106
0.99118
0.99078
0.99154
0.99286
0.99082
0.99212
0.9923
0.9912
0.99058
0.99138
0.99246
0.99234
0.99206
0.9911
0.9926
0.99292
0.99158
0.99262
0.99344
0.99206
0.9925
0.99306
0.99334
0.99274
0.99106
0.99408


In [23]:
accuracy = 0
num = 0
model.eval()
with torch.no_grad():
    for X,Y in batches(x_test,y_test):
        X = torch.Tensor(X.transpose(0, 3, 1, 2)) # Convert X to a PyTorch Tensor
        Y = torch.Tensor(Y)
        X = X.to(device)
        Y = Y.to(device)
        prediction = model(X)
        accuracy += (prediction.max(1)[1] == Y).sum().item()
        num += Y.size(0)

    print(accuracy/num)

0.8076
